In [10]:
!pip install git+https://github.com/JerzyKultura/Agora.git
!pip install openai pyyaml
!pip install opentelemetry-api opentelemetry-sdk
!pip install opentelemetry-instrumentation-openai
!pip install fastmcp openai

  Cloning https://github.com/JerzyKultura/Agora.git to /tmp/pip-req-build-y05c4yn4
  Running command git clone --filter=blob:none --quiet https://github.com/JerzyKultura/Agora.git /tmp/pip-req-build-y05c4yn4
  Resolved https://github.com/JerzyKultura/Agora.git to commit 47229c1ea5fbbc2215b874b69bc3660be8543541
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [11]:
import os

In [12]:
from agora.instrument import init_otel
from agora.telemetry import AuditLogger, AuditedAsyncFlow, AuditedAsyncNode
from openai import OpenAI
import asyncio
import os

In [13]:
# Set your OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = "..."  # Replace with your key

# Initialize OpenTelemetry with console output
init_otel("agora-chat-demo")

✅ OpenTelemetry initialized for agora-chat-demo


In [14]:
class SendPrompt(AuditedAsyncNode):
    """Node that sends a prompt to OpenAI and gets a response"""

    async def exec_async(self, prep_res):
        # Create OpenAI client
        client = OpenAI()

        # Make API call
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": "Say hello briefly"}]
        )

        # Extract and return the response text
        return response.choices[0].message.content


class PrintResponse(AuditedAsyncNode):
    """Node that prints the model's response"""

    async def exec_async(self, text):
        print("\n" + "="*50)
        print("Model said:", text)
        print("="*50 + "\n")
        return text

In [15]:
class ChatFlow(AuditedAsyncFlow):
    """Simple chat flow: send prompt -> print response"""

    def __init__(self, logger):
        super().__init__("chat_flow", logger)

        # Create nodes
        send = SendPrompt("send_prompt", logger)
        show = PrintResponse("print_response", logger)

        # Build flow
        self.start(send)
        send >> show

In [16]:

async def main():
    """Main async function to run the workflow"""

    print("\n🚀 Starting Agora Chat Flow with Hierarchical Tracing\n")

    # Create audit logger with custom save directory
    logger = AuditLogger(save_dir="./traces")

    # Create and run flow
    flow = ChatFlow(logger)
    result = await flow._run_async({})

    # Save trace data to JSON
    print("\n📊 Saving trace data...")
    logger.save_trace_json("trace.json")

    # Display summary
    print("\n📈 Audit Summary:")
    summary = logger.get_summary()
    print(f"  - Total events: {summary['total_events']}")
    print(f"  - Duration: {summary['duration_seconds']:.2f} seconds")
    print(f"  - Event types: {summary['event_counts']}")

    print("\n✅ Complete! Check ./traces/trace.json for hierarchical trace data")

    return result



In [17]:
result = await main()



🚀 Starting Agora Chat Flow with Hierarchical Tracing

{
    "name": "node.send_prompt",
    "context": {
        "trace_id": "0xf7c2d5d1aa3ea315dba030cd898b6ba5",
        "span_id": "0x00aaf6ef3d0f79d4",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0xed418f5196c73da5",
    "start_time": "2025-10-07T19:26:18.738023Z",
    "end_time": "2025-10-07T19:26:20.352667Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "node_name": "send_prompt",
        "node_type": "SendPrompt",
        "latency_ms": "1614.5331859588623",
        "result_type": "NoneType",
        "output_batch_size": "6"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "telemetry.sdk.language": "python",
            "telemetry.sdk.name": "opentelemetry",
            "telemetry.sdk.version": "1.37.0",
            "service.name": "agora-chat-demo"
        },
        "schema_url": ""
    }
}

Model said: No